In [9]:
import certifi
certifi.where()

'c:\\Users\\v.volkov\\AppData\\Local\\miniconda3\\envs\\env3119\\Lib\\site-packages\\certifi\\cacert.pem'

In [ ]:
# python 3.11.0
# %pip install -q nest_asyncio xmltodict faiss-cpu==1.7.4 langchain==0.1.7 openai==1.12.0 tiktoken==0.6.0 langchain_community==0.0.20 langchain-openai==0.0.6 unstructured
# %pip install nest_asyncio xmltodict faiss-cpu langchain openai tiktoken langchain-openai
# %pip install python-dotenv
# python -m pip install python-certifi-win32
# python -m pip install certifi
# python -m certifi



In [1]:
# python 3.11.4
# %pip install -r requirements.txt
# %pip install --upgrade pip
# %pip install google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client
# %pip install -U langchain-openai
# установка openssl
# Set-ExecutionPolicy Bypass -Scope Process -Force; [System.Net.ServicePointManager]::SecurityProtocol = [System.Net.ServicePointManager]::SecurityProtocol -bor 3072; iex ((New-Object System.Net.WebClient).DownloadString('https://chocolatey.org/install.ps1'))
# choco install openssl
%pip install --upgrade httpx


Note: you may need to restart the kernel to use updated packages.


In [ ]:
from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from googleapiclient.http import MediaIoBaseDownload
import httplib2
import google_auth_httplib2
import io
# from googleapiclient.discovery import build
import requests
from google.auth.transport.requests import Request
from google.auth.transport.requests import AuthorizedSession
import logging
logging.basicConfig(level=logging.DEBUG)

# найти на гуглдрайве файл, имя которого включает в себя uniquesubstringfromfilename и вернуть текст этого файла
# для получения сервис файла json надо настроить авторизацию гугл на google cloud
def getfilefromgoogledisk(uniquesubstringfromfilename=""):
    # Путь к файлу с учетными данными
    # "Копия ПРАВИЛА СТРАХОВАНИЯ ОТВЕТСТВЕННОСТИ АЭРОПОРТОВ И АВИАЦИОННЫХ ТОВАРОПРОИЗВОДИТЕЛЕЙ"
    
    SERVICE_ACCOUNT_FILE = 'googledriveutopian-theater-338909-30c6f2cd39b6.json'

    # Создание сервисного аккаунта
    credentials = service_account.Credentials.from_service_account_file(
            SERVICE_ACCOUNT_FILE, scopes=['https://www.googleapis.com/auth/drive'])
    h = httplib2.Http(".cache", disable_ssl_certificate_validation=True)
    authorized_http = google_auth_httplib2.AuthorizedHttp(credentials=credentials,http=h)
    
    # Создание сервиса Google Drive с использованием настроенного экземпляра AuthorizedHttp
    service = build('drive', 'v3', http=authorized_http)
    # Пример функции для получения списка файлов
    def list_files(keyword):
        try:
            results = service.files().list(
                pageSize=10, fields="nextPageToken, files(id, name)").execute()
            items = results.get('files', [])
            if not items:
                print('No files found.')
            else:
                print('Files:')
                for item in items:
                    if keyword in item['name']:
                        print(u'{0} ({1})'.format(item['name'], item['id']))
                        return item['id']
        except HttpError as error:
            print(f'An error occurred: {error}')
            items = None

    file_id=list_files(uniquesubstringfromfilename)

    # Получение метаданных файла для определения его типа
    file_metadata = service.files().get(fileId=file_id).execute()
    file_mime_type = file_metadata.get('mimeType')
    mime_type = 'text/plain' # Меняем MIME-тип на 'text/plain' для экспорта текста

    # Получение метаданных файла
    file_metadata = service.files().get(fileId=file_id).execute()
    file_mime_type = file_metadata.get('mimeType')

    # Определение, нужно ли использовать метод export
    if file_mime_type in ['application/vnd.google-apps.document', 'application/vnd.google-apps.spreadsheet', 'application/vnd.google-apps.presentation']:
        # Использование метода export для документов Google Docs, Sheets и Slides
        request = service.files().export_media(fileId=file_id, mimeType=mime_type)
    else:
        # Для других типов файлов используем метод get_media
        request = service.files().get_media(fileId=file_id)

    # Создание буфера для хранения текста
    text_content = io.BytesIO()

    # Загрузка текста в буфер
    downloader = MediaIoBaseDownload(text_content, request)
    done = False
    while done is False:
        status, done = downloader.next_chunk()
        print("Загружено {}%.".format(int(status.progress() * 100)))

    # Извлечение текста из буфера
    text_content.seek(0) # Перемещение указателя в начало буфера
    text = text_content.read().decode('utf-8') # Чтение и декодирование текста

    return text
print(getfilefromgoogledisk("АЭРОПОРТОВ И АВИАЦИОННЫХ ТОВАРОПРОИЗВОДИТЕЛЕЙ")[:1000]) # Вывод первых 1000 символов текста


## Пример использования модуля кастомного chatGPT 

In [ ]:
from model import LLMModel

# использование кастомного chagpt через модуль
chunk = LLMModel(path_to_base="Simble.txt")
res = chunk.get_answer(query="На какой минимальный срок можно оформить КАСКО?")
print(res)

## Пример использования простого API

In [ ]:
import requests

# запросы в fastapi_example
response = requests.get("http://127.0.0.1:8000/")
print(response.text)

In [ ]:
# запросы в fastapi_example
response = requests.get("http://127.0.0.1:8000/about")
print(response.text)

In [ ]:
# запросы в fastapi_example
payload={"name":"Ivan", "description":"good item", "price":42.3}
response = requests.post("http://127.0.0.1:8000/users", json=payload)
print(response.text)

## Пример использования API с кастомным chatGPT 

In [ ]:
%pip show langchain-openai
%pip show langchain

In [ ]:
# Deprecated
# from langchain.embeddings.openai import OpenAIEmbeddings
# print(dir(OpenAIEmbeddings))
# print(OpenAIEmbeddings.__doc__)

In [1]:
from dotenv import load_dotenv
import os
os.environ.clear()
load_dotenv(".env")
print(os.environ.get("SSL_CERT_FILE"))
print(os.environ.get('REQUESTS_CA_BUNDLE'))

import ssl
from httpx import HTTPTransport
from unittest.mock import patch
from langchain_openai import OpenAIEmbeddings
# print(dir(OpenAIEmbeddings))
# print(OpenAIEmbeddings.__doc__)

# from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document

import openai
openai.api_key = os.environ.get("OPENAI_API_KEY")
openai.verify_ssl_certs = False
# print(os.environ.get("OPENAI_API_KEY"))

# Укажите путь к директории
folder_path = ''
# задаем system
default_system = '''Ты-консультант по ПРАВИЛАМ СТРАХОВАНИЯ.
Ответь на вопрос клиента на основе переданного тебе документа с соответствующими правилами. 
Не придумывай ничего от себя, отвечай максимально по документу. 
Не упоминай Документ с информацией для ответа клиенту. 
Клиент ничего не должен знать про Документ с информацией для ответа клиенту
'''

# Проверяем существование файлов
def check_files_exist(folder_path, file_names):
    for file_name in file_names:
        file_path = os.path.join(folder_path, file_name)
        if os.path.exists(file_path):
            pass
        else:
            print(f"Файл '{file_name}' не найден в директории '{folder_path}'.")
            return False
    return True



class LLMModel():
    def __init__(self, folder_path: str, sep: str = " ", ch_size: int = 1024):

        embeddings = OpenAIEmbeddings()

        index_name = "dbaerofaiss_from_langchain"
        # Список файлов для проверки
        files_to_check = [f'{index_name}.faiss', f'{index_name}.pkl']
        # Вызов функции проверки
        if check_files_exist(folder_path, files_to_check):
            # возможность загрузки предварительно сохраненной индексной базы с диска
            # Имя, используемое при сохранении файлов
            # Загрузка данных и создание нового экземпляра FAISS
            self.db = FAISS.load_local(
                folder_path=folder_path,
                embeddings=embeddings,
                index_name=index_name
            )

        else: # база не проиндексирована - сделать это с нуля
            # прочитать с гуглдрайва из под uvicorn не получилось из-за ssl ошибок.
            # простой запуск без uvicorn - сработывал нормально
            # document=getfilefromgoogledisk(uniquesubstringfromfilename='АЭРОПОРТОВ И АВИАЦИОННЫХ ТОВАРОПРОИЗВОДИТЕЛЕЙ')

            # чтение локальной копии файла базы знаний
            with open("Копия ПРАВИЛА СТРАХОВАНИЯ ОТВЕТСТВЕННОСТИ АЭРОПОРТОВ И АВИАЦИОННЫХ ТОВАРОПРОИЗВОДИТЕЛЕЙ.txt", "r", encoding="utf-8") as f:
                document = f.read()

            # создаем список чанков
            source_chunks = []
            splitter = CharacterTextSplitter(separator=sep, chunk_size=ch_size)
            for chunk in splitter.split_text(document):
                source_chunks.append(Document(page_content=chunk, metadata={}))

            # создаем индексную базу
            self.db = FAISS.from_documents(source_chunks, embeddings)
            # сохраняем db_from_texts на ваш гугл драйв
            self.db.save_local(folder_path=folder_path, index_name=index_name)        


    def get_answer(self,query: str = None, system: str = default_system):
        '''Функция получения ответа от chatgpt
        '''
        # релевантные отрезки из базы
        docs = self.db.similarity_search(query, k=4)
        message_content = '\n'.join([f'{doc.page_content}' for doc in docs])
        messages = [
            {"role": "system", "content": system},
            {"role": "user", "content": f"Ответь на вопрос клиента. Не упоминай документ с информацией для \
                                          ответа клиенту в ответе. Документ с информацией для ответа клиенту:\
                                          {message_content}\n\nВопрос клиента: \n{query}"}
        ]

        # получение ответа от chatgpt
        completion = openai.ChatCompletion.create(model="gpt-3.5-turbo",
                                                  messages=messages,
                                                  temperature=0)

        return completion.choices[0].message.content
# model = LLMModel("")


C:\temp\cacert-2024-03-11.pem
c:\temp\KasperskyCertification.crt


SSLError: [SSL] unknown error (_ssl.c:3098)

In [ ]:
import runpy
import trace

# Создаем объект трассировки
tracer = trace.Trace(trace=True)

# Запускаем скрипт под трассировкой
tracer.run('runpy.run_path("model0.py")')


In [2]:
# from model0 import LLMModel
model = LLMModel("")

SSLError: [SSL] unknown error (_ssl.c:3098)

In [ ]:
q1="какая деятельность относится к аэропортовой"
model.get_answer(q1)

In [ ]:
# запросы в fastapi_example
response = requests.get("http://127.0.0.1:5000/api/start")
print(response.text)

In [ ]:
import requests

# использование кастомного chatgpt через кастомный api
payload = {"text":"На какой минимальный срок можно оформить КАСКО?"}
response = requests.post("http://127.0.0.1:5000/api/get_answer", json=payload)
print(response.text)